## Use package bigsnpr to create PCA and remove outliers by using robust mahalanobis distance

In [89]:
library(bigsnpr)

## Do some QC with plink within bigsnpR

In [ ]:
bedfile <- "/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb28374_exomedata/exome_data_OCT2020/ukb23155_c1_b0_v1.bed"
prefix <- sub_bed(bedfile)
plink <- "/home/dc2325/software/plink"

In [ ]:
bedfile <- system.file("extdata", "example.bed", package = "bigsnpr")
prefix  <- sub_bed(bedfile)

In [ ]:
snp_plinkQC(
  plink.path = plink,
  prefix.in = prefix,
  file.type = "--bfile",
  prefix.out = paste0(prefix.in, "_QC"),
  maf = 0.01,
  geno = 0.1,
  mind = 0.1,
  hwe = 1e-50,
  autosome.only = TRUE,
  extra.options = "",
  verbose = TRUE
)

## Calculate relatedness using Plink2 - KING

In [98]:
bedfile <- download_1000G("data")
plink2 <- download_plink2("data")
rel <- snp_plinkKINGQC_2(
  plink2.path = plink2,
  bedfile.in = bedfile,
  thr.king = 2^-4.5,
  make.bed = FALSE,
  ncores = 2
)
str(rel)

'data.frame':	31 obs. of  8 variables:
 $ FID1   : int  0 0 0 0 0 0 0 0 0 0 ...
 $ IID1   : chr  "HG00120" "HG00240" "HG00542" "HG00595" ...
 $ FID2   : int  0 0 0 0 0 0 0 0 0 0 ...
 $ IID2   : chr  "HG00116" "HG00238" "HG00475" "HG00584" ...
 $ NSNP   : int  1664852 1664852 1664852 1664852 1664852 1664852 1664852 1664852 1664852 1664852 ...
 $ HETHET : num  0.111 0.1105 0.1024 0.101 0.0992 ...
 $ IBS0   : num  0.0333 0.0367 0.0302 0.037 0.0367 ...
 $ KINSHIP: num  0.0821 0.068 0.0854 0.0541 0.0535 ...


In [97]:
snp_plinkKINGQC_2 <- function (plink2.path, bedfile.in, bedfile.out = NULL, thr.king = 2^-3.5, 
    make.bed = TRUE, ncores = 1, extra.options = "", verbose = TRUE) 
{
    v <- system(paste(plink2.path, "--version"), intern = TRUE)
    if (substr(v, 1, 8) != "PLINK v2") 
        stop2("This requires PLINK v2; got '%s' instead.", v)
    prefix.in <- sub_bed(bedfile.in)
    if (make.bed) {
        if (is.null(bedfile.out)) 
            bedfile.out <- paste0(prefix.in, "_norel.bed")
        assert_noexist(bedfile.out)
        bigsnpr:::system_verbose(paste(plink2.path, "--bfile", prefix.in, 
            "--make-bed --king-cutoff", thr.king, "--out", sub_bed(bedfile.out), 
            "--threads", ncores, extra.options), verbose = verbose)
        bedfile.out
    }
    else {
        prefix.out <- tempfile()
        bigsnpr:::system_verbose(paste(plink2.path, "--bfile", prefix.in, 
            "--make-king-table --king-table-filter", thr.king, 
            "--out", prefix.out, "--threads", ncores, extra.options), 
            verbose = verbose)
        rel_df <- bigreadr::fread2(paste0(prefix.out, ".kin0"), 
            header = TRUE, nThread=ncores)
        names(rel_df) <- sub("^#(.*)$", "\\1", names(rel_df))
        rel_df
    }
}

In [94]:
bigsnpr:::system_verbose

function (..., verbose) 
{
    system(..., ignore.stdout = !verbose, ignore.stderr = !verbose)
}
<bytecode: 0x556db980cb30>
<environment: namespace:bigsnpr>

## Calculate PCA

### 1. Remove related individuals from calculations

In [10]:
rel <- read.table("/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb28374_exomedata/exome_data_OCT2020/ukb_rel_a32285_s488244.dat", header=T)
head(rel)
dim(rel)
summary(rel$Kinship)

,ID1,ID2,HetHet,IBS0,Kinship
,<int>,<int>,<dbl>,<dbl>,<dbl>
1,1000019,4020065,0.067,0.0000,0.2487
2,1000035,3287226,0.047,0.0138,0.0714
3,1000054,1291619,0.051,0.0102,0.1154
4,1000170,4380447,0.044,0.0146,0.0543
5,1000224,2776529,0.078,0.0042,0.2583
6,1000291,3042243,0.048,0.0125,0.0851


[1] 107126      5

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-1.0000  0.0566  0.0699  0.1183  0.2254  0.4999 

In [14]:
fam <- read.table("/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb28374_exomedata/exome_data_OCT2020/ukb23155_s200631.fam")
colnames(fam) <- c('FID', 'IID', 'father', 'mother', 'sex', 'pheno')
head(fam)

,FID,IID,father,mother,sex,pheno
,<int>,<int>,<int>,<int>,<int>,<int>
1,1434748,1434748,0,0,2,-9
2,5523981,5523981,0,0,1,-9
3,5023838,5023838,0,0,2,-9
4,4023729,4023729,0,0,1,-9
5,4442146,4442146,0,0,2,-9
6,5654789,5654789,0,0,2,-9


In [22]:
ind.rel <- match(c(rel$ID1, rel$ID2), fam$IID)  # /!\ use $ID1 instead with old PLINK
ind.rel
#ind.norel <- (fam)[-ind.rel]

[1]  21649  43163     NA     NA  40083 106795     NA     NA 166565 133843
   [11]     NA 182865 182865  19719 100921     NA  42427 109414     NA  20899
   [21]     NA     NA     NA     NA     NA     NA  27989  48365  48365 107510
   [31]     NA  20599     NA     NA     NA     NA     NA  94073 134117  24150
   [41]     NA  85209     NA     NA 128573 142299     NA     NA     NA  57796
   [51] 187254 187254     NA     NA     NA  92043  19411     NA     NA     NA
   [61]  56774     NA     NA 157514     NA     NA  82053  79648     NA  77686
   [71]     NA     NA     NA     NA     NA     NA     NA     NA     NA     NA
   [81]     NA     NA     NA     NA     NA     NA     NA     NA     NA     NA
   [91]     NA     NA     NA     NA     NA     NA     NA     NA  88600     NA
  [101]     NA     NA 180377     NA     NA  53532  26698  49592     NA 132397
  [111]  52262 176348     NA 184473    226 156816 146934 118518   2249  83787
  [121]     NA     NA     NA     NA  83708     NA     NA     NA     NA  94382
  [131]  52808     NA  30952     NA     NA  54725     NA     NA     NA     NA
  [141] 118174     NA     NA     NA   9185     NA     NA     NA     NA     NA
  [151]     NA 198349     NA     NA     NA     NA     NA     NA 128698  43411
  [161]     NA     NA 164644     NA     NA  34176  34176     NA     NA     NA
  [171]  82468   8487   8487 108348     NA  63332 181659  10560     NA     NA
  [181]     NA  70075  55714     NA     NA     NA 174142  90609     NA  23579
  [191]     NA 147382     NA     NA     NA 171698     NA     NA     NA     NA
  [201]     NA  37976     NA  64285     NA     NA     NA     NA 128957 128957
  [211]     NA     NA     NA  91037  91037  36280 121728     NA  96797     NA
  [221] 168104 103029     NA  47497  96107     NA     NA     NA    880    880
  [231]     NA     NA     NA 170912     NA  75914 149426 117750     NA 150394
  [241]     NA     NA     NA  59108  26072 152138 168061     NA     NA     NA
  [251]     NA     NA     NA     NA  70977  70977 102872     NA  88855     NA
  [261]     NA     NA 114049     NA 100856     NA     NA     NA     NA 182932
  [271] 187419     NA  87064  87064 185621 185621     NA     NA     NA  76795
  [281]     NA    918   9750 114558 114558 114558 114558     NA   2367 131447
  [291]     NA     NA     NA     NA     NA     NA     NA     NA     NA     NA
  [301] 172013 172013 172013     NA 101267 101267     NA     NA     NA     NA
  [311]     NA  63313     NA  81059     NA     NA     NA   8017   8017     NA
  [321]  12578  12578     NA     NA     NA     NA     NA     NA     NA     NA
  [331]     NA     NA     NA 145810     NA 160518   3052     NA     NA 114539
  [341]     NA     NA     NA     NA     NA     NA     NA     NA  43253     NA
  [351]     NA 128761 128761     NA  14077  29659  18664  18664  18664     NA
  [361]     NA     NA   8733 152740  80136     NA     NA     NA     NA     NA
  [371]     NA 186694 186694  44891     NA     NA     NA 168348 168348 168348
  [381]     NA     NA     NA  59323  53811     NA     NA     NA     NA  69100
  [391]     NA   5774     NA  53230     NA 138589  82681 194898     NA     NA
  [401]     NA     NA     NA     NA     NA     NA     NA     NA     NA     NA
  [411]     NA     NA     NA     NA 181689     NA 115737     NA     NA     NA
  [421]     NA     NA     NA     NA     NA     NA     NA     NA     NA     NA
  [431]     NA 106173 114201     NA     NA     NA  34497     NA     NA     NA
  [441]     NA 113587 121207     NA 138121     NA  15078  43450  43450  43450
  [451]     NA 186593  60773     NA 124865     NA     NA     NA     NA  64624
  [461]     NA     NA 119239     NA 150779 109158 144449     NA     NA     NA
  [471]     NA  25598  45645     NA  52222     NA     NA     NA     NA     NA
  [481]  39019     NA     NA  87812     NA     NA     NA     NA     NA     NA
  [491]     NA 172494  76747     NA 176638 176638 118354     NA     NA     NA
  [501] 124076     NA     NA     NA 129816 164754 164754 164754   2246  80760
  [511]   8278     NA     

In [13]:
library(dplyr)
related <- rel %>%
    filter(ID1 %in% exomed_ind$IID | ID2 %in% exomed_ind$IID)
head(related)
dim(related)

,ID1,ID2,HetHet,IBS0,Kinship
,<int>,<int>,<dbl>,<dbl>,<dbl>
1,1000019,4020065,0.067,0.0000,0.2487
2,1000035,3287226,0.047,0.0138,0.0714
3,1000054,1291619,0.051,0.0102,0.1154
4,1000224,2776529,0.078,0.0042,0.2583
5,1000291,3042243,0.048,0.0125,0.0851
6,1000516,1748770,0.080,0.0046,0.2611


[1] 68532     5

In [3]:
king <- read.table("/home/dc2325/scratch60/pca/genotypes21_22.kin0", header=F)
colnames(king) <- c("FID1","ID1","FID2","ID2","NSNP","HETHET","IBS0","KINSHIP")
head(king)

,FID1,ID1,FID2,ID2,NSNP,HETHET,IBS0,KINSHIP
,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>
1,1000022,1000022,1000019,1000019,16507,0.0687587,0.0383474,-0.02847260
2,1000035,1000035,1000019,1000019,16526,0.0743677,0.0418129,-0.03907170
3,1000035,1000035,1000022,1000022,16476,0.0759286,0.0399976,-0.01657660
4,1000046,1000046,1000019,1000019,16547,0.0710099,0.0388590,-0.02963620
5,1000046,1000046,1000022,1000022,16497,0.0741347,0.0338849,0.01114420
6,1000046,1000046,1000035,1000035,16516,0.0730806,0.0377815,-0.00886376


In [20]:
library(dplyr)
rel <- king %>%
    filter (KINSHIP > 0.055)
rel

FID1,ID1,FID2,ID2,NSNP,HETHET,IBS0,KINSHIP
<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>
1001937,1001937,1000210,1000210,16534,0.0835249,0.0273376,0.0595560
1001937,1001937,1000584,1000584,16515,0.0822283,0.0267030,0.0631950
1002650,1002650,1001941,1001941,16480,0.0825850,0.0284587,0.0573131
1004369,1004369,1000141,1000141,16529,0.0814326,0.0278904,0.0550962
1004369,1004369,1000584,1000584,16504,0.0831920,0.0289021,0.0558927
1004560,1004560,1004369,1004369,16538,0.0838675,0.0283589,0.0592283


In [31]:
rel_1 <- sort(unique(unlist(rel[, c("ID1", "ID2")])))
rel_1
#FID <- sort(rel_1)

[1] 1000141 1000210 1000584 1001937 1001941 1002650 1004369 1004560

In [30]:
rel_3 <- data.frame(FID)
rel_3$IID <- rel_3$FID
head(rel_3)

,FID,IID
,<int>,<int>
1,1000141,1000141
2,1000210,1000210
3,1000584,1000584
4,1001937,1001937
5,1001941,1001941
6,1002650,1002650


In [60]:
bedfile <- ("/home/dc2325/scratch60/pca/asian_ancestry/ukb23155_s200631.filtered.merged.prune.bed")

In [ ]:
(obj.bed <- bed(bedfile))

In [ ]:
bed

In [ ]:

ind.rel <- match(c(rel$IID1, rel$IID2), obj.bed$fam$sample.ID)  # /!\ use $ID1 instead with old PLINK
ind.norel <- rows_along(obj.bed)[-ind.rel]

obj.svd <- bed_autoSVD(obj.bed, ind.row = ind.norel, k = 20,
                       ncores = nb_cores())